# 라이블러리 불러오기

In [2]:
import openai
import os
from dotenv import load_dotenv, find_dotenv
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

# 모델 불러오기

In [3]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo")

# 프롬프트 템플릿 맛보기

- 프롬프트 템플릿은 크게 2가지 존재
    1. PromptTemplate : 일반적인 프롬프트 템플릿 생성
    2. ChatPromptTemplate : 채팅 LLM에 프롬프트를 전달하는 데 특화된 템플릿

In [5]:
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [12]:
chat_prompt = ChatPromptTemplate.from_template("tell me a joke about {subject}")
chat_prompt_value = chat_prompt.format_prompt(subject='soccer')
chat_prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me a joke about soccer')])

In [14]:
print(chat_prompt_value.to_string())

Human: tell me a joke about soccer


# 프롬프트 템플릿 활용
- 반복적인 프롬프트 삽입 시, Prompt Template 를 통해 간편히 LLM 활용 가능
- GPT-3와 프롬프트 템플릿 활용해 보기

In [15]:
from langchain.prompts.prompt import PromptTemplate

template = """
너는 요리사야. 내가 가진 재료들을 가지고 만들수 있는 요리를 추천해 주고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아

<재료>
{재료}
"""

prompt_template = PromptTemplate(
    input_variables = ["재료"],
    template=template
)

In [16]:
print(prompt_template.format(재료='양파,계란,사과, 빵'))


너는 요리사야. 내가 가진 재료들을 가지고 만들수 있는 요리를 추천해 주고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아

<재료>
양파,계란,사과, 빵



In [10]:
llm.predict(prompt_template.format(재료='양파, 계란, 사과, 빵'))

'네가 가진 재료로 만들 수 있는 요리 중 하나는 "양파 계란말이"야. 요리의 레시피를 제시해 줄게.\n\n[양파 계란말이 레시피]\n1. 양파를 적당한 크기로 다지고, 계란을 잘 풀어서 볼에 넣는다.\n2. 팬에 식용유를 두르고 양파를 볶다가 약간의 소금과 후춧가루로 간을 한다.\n3. 양파가 투명해질 때까지 볶은 후에 풀어둔 계란을 넣고 섞는다.\n4. 계란이 익을 때까지 볶으면서 젓가락이나 주걱으로 뒤집어준다.\n5. 양파와 계란이 골고루 섞이고 익으면, 접시에 옮겨 담아서 완성한다.\n6. 빵과 함께 내놓으면 맛있는 양파 계란말이가 완성되었어.\n\n재료들을 활용하여 맛있는 요리를 만들어보길 바래!'

# ChatGPT와 프롬프트 템플릿 활용

In [12]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import AIMessage, HumanMessage, SystemMessage

In [14]:
llm = ChatOpenAI(temperature=0)

template = """
너는 요리사야. 내가 가진 재료들을 가지고 만들수 있는 요리를 추천해 주고, 그 요리의 레시피를 제시해줘.
내가 가진 재료는 아래와 같아

<재료>
{재료}
"""

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = '{재료}'
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

answer = llm(chat_prompt.format_prompt(재료='양파, 계란, 사과, 빵').to_messages())
answer.content

'가지고 있는 재료로 만들 수 있는 요리 중 하나는 "양파 계란말이"입니다. 이 요리는 양파와 계란을 함께 볶아서 만드는 간단하고 맛있는 요리입니다. 아래는 양파 계란말이의 레시피입니다.\n\n[양파 계란말이 레시피]\n1. 양파를 깍두기 형태로 썰어줍니다.\n2. 팬에 식용유를 두르고 양파를 넣어 중간 불에서 볶아줍니다.\n3. 양파가 투명해질 때까지 볶은 후 계란을 풀어 넣어줍니다.\n4. 계란이 익을 때까지 저어가며 볶아줍니다.\n5. 양파 계란말이가 완성되면 빵과 함께 즐기세요.\n\n이렇게 간단하게 양파 계란말이를 만들 수 있습니다. 재료들을 활용하여 맛있는 요리를 즐기세요!'

# Few-shot 예제를 통한 프롬프트 템플릿
- Few-shot 이란, 딥러닝 모델이 결과물을 출력할 떄 예시 결과물을 제시함으로써 원하는 결과물로 유도하는 방법론
- LLM역시, Few-shot 예제를 제공하면 예제와 유사한 형태의 결과물 출력
- 내가 원하는 출력의 형태가 특수하거나, 구조화된 답변을 원할 경우, 결과물의 예시를 수 개 제시하며 결과물 품질 향상

In [7]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [8]:
examples = [
    {
        "question":"아이유로 삼행시 만들어줘",
        "answer":"""
        아 : 아이유는
        이 : 이런 강의를 들을 이
        유 : 유가 없다
        """
    },
    {
        "question":"김민수로 삼행시 만들어줘",
        "answer":"""
        김 : 김치는 맛있다
        민 : 민달팽이도 좋아하는 김치!
        수 : 수억을 줘도 김치는 내꺼!
        """
    }
]

In [9]:
examples[0]

{'question': '아이유로 삼행시 만들어줘',
 'answer': '\n        아 : 아이유는\n        이 : 이런 강의를 들을 이\n        유 : 유가 없다\n        '}

In [10]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question : {question}\n{answer}")
example_prompt.format(**examples[0])

'Question : 아이유로 삼행시 만들어줘\n\n        아 : 아이유는\n        이 : 이런 강의를 들을 이\n        유 : 유가 없다\n        '

In [11]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=["input"]
)

In [28]:
prompt.format(input="호날두로 삼행시 만들어줘")

'Question : 아이유로 삼행시 만들어줘\n\n        아 : 아이유는\n        이 : 이런 강의를 들을 이\n        유 : 유가 없다\n        \n\nQuestion : 김민수로 삼행시 만들어줘\n\n        김 : 김치는 맛있다\n        민 : 민달팽이도 좋아하는 김치!\n        수 : 수억을 줘도 김치는 내꺼!\n        \n\nQuestion: 호날두로 삼행시 만들어줘'

In [30]:
llm.predict("호날두로 삼행시 만들어줘")

'호날두는 축구의 신\n공을 다루면 빛나는 스타\n승리를 위해 노력하는 모습이 인상깊어'

In [33]:
llm.predict(prompt.format(input="호날두로 삼행시 만들어줘"))

'호 : 호날두는 축구의 신\n날 : 날개 달린 듯이 뛰어난 실력\n두 : 두 번째로 세계를 놀라게 하는 선수'

# Example Selector를 이용한 동적 Few-shot 러닝
- Few_shot 예제를 동적으로 입력시 사용

In [22]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

# These are a lot of examples of a pretend task of creating antonyms.
examples = [
    {"input": "행복", "output": "슬픔"},
    {"input": "흥미", "output": "지루"},
    {"input": "불안", "output": "안정"},
    {"input": "긴 기차", "output": "짧은 기차"},
    {"input": "큰 공", "output": "작은 공"},
]

In [45]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, OpenAIEmbeddings(), Chroma, k=1
)

In [46]:
similar_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘",
    suffix="Input: {단어}\nOutput:",
    input_variables=["단어"],
)

In [47]:
similar_prompt.format(단어="무서운")

'주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘\n\nInput: 불안\nOutput: 안정\n\nInput: 무서운\nOutput:'

In [48]:
similar_prompt.format(단어='큰 비행기')

'주어진 입력에 대해 반대의 의미를 가진 단어를 출력해줘\n\nInput: 긴 기차\nOutput: 짧은 기차\n\nInput: 큰 비행기\nOutput:'

In [49]:
query = "큰 비행기"

llm.predict(similar_prompt.format(단어=query))

'작은 비행기'

# Output Parser를 활용한 출력값 조정
- LLM의 답변을 내가 원하는 형태로 고정하고 싶을때 OutputParser 사용
- list, json 등의 형태

In [50]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [51]:
output_parser = CommaSeparatedListOutputParser()

In [52]:
format_instructions = output_parser.get_format_instructions()
format_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [53]:
prompt = PromptTemplate(
    template="{주제}5개를 추천해줘. \n{format_instructions}",
    input_variables=['주제'],
    partial_variables={"format_instructions":format_instructions}
)

In [55]:
_input = prompt.format(주제="영화")
_input

'영화5개를 추천해줘. \nYour response should be a list of comma separated values, eg: `foo, bar, baz`'

In [58]:
output = llm.predict(_input)
output

'어벤져스: 엔드게임, 인셉션, 레옹, 쇼생크 탈출, 어메이징 스파이더맨'

In [59]:
output_parser.parse(output)

['어벤져스: 엔드게임', '인셉션', '레옹', '쇼생크 탈출', '어메이징 스파이더맨']